## Export Cat to Image

In [2]:
!pip install cvlib opencv-python tensorflow 

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
  Using cached cvlib-0.2.7-py3-none-any.whl
  Using cached tensorflow-2.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (489.8 MB)
  Using cached progressbar-2.5-py3-none-any.whl
  Using cached imageio-2.31.5-py3-none-any.whl (313 kB)
  Using cached imutils-0.5.4-py3-none-any.whl
  Using cached libclang-16.0.6-py2.py3-none-manylinux2010_x86_64.whl (22.9 MB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached protobuf-4.24.4-cp37-abi3-manylinux2014_x86_64.whl (311 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tensorflow_io_gcs_filesystem-0.34.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.4 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import cvlib as cv
from cvlib.object_detection import draw_bbox
import cv2
import os

class Extract:

    def __init__(self, source, folder):
        self.video = cv2.VideoCapture(source)
        if not self.video.isOpened():
            print("Could not open video")
            exit()
        self.folder = folder
        if not os.path.exists(self.folder):
            os.makedirs(self.folder)
        self.frame_count = 0

    def export_images(self, label):
        while self.video.isOpened():
            status, frame = self.video.read()
            if not status:
                break

            bbox, detected_labels, conf = cv.detect_common_objects(frame, confidence=0.25, model='yolov4-tiny')

            for l, c in zip(detected_labels, conf):
                if l == label:
                    idx = detected_labels.index(l)
                    x, y, x2, y2 = map(int, bbox[idx])

                    if x >= 0 and y >= 0 and x2 >= 0 and y2 >= 0:
                        cropped = frame[y:y2, x:x2]

                        if cropped.any():
                            output_path = os.path.join(self.folder, f"{label}_{self.frame_count}.jpg")
                            cv2.imwrite(output_path, cropped)
                            self.frame_count += 1

            out = draw_bbox(frame, bbox, detected_labels, conf, write_conf=True)
            cv2.imshow("Real-time object detection", out)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        self.video.release()
        cv2.destroyAllWindows()



2023-10-21 19:16:43.196913: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-21 19:16:43.233478: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-21 19:16:43.233508: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-21 19:16:43.233528: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-21 19:16:43.248487: I tensorflow/core/platform/cpu_feature_g

In [27]:
# Uso da classe Extract
path = "../../Documentos/dataset/Ada/ada.mp4"
folder = "ada"
extract = Extract(path, folder)

extract.export_images(label="cat")


## Upload New Label to Dataset

In [ ]:
# !aws s3 cp ada/ s3://BUCKET/gatos/dataset/ada/ --recursive
